In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

from BayesLeNet5 import BayesLeNet5

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 1024
val_dataset = MNIST(
    root="data", 
    download=True, 
    train=False, 
    transform=transforms.ToTensor()
)
val_loader = DataLoader(
    dataset=val_dataset, 
    batch_size=batch_size, 
    shuffle=False
)

model = BayesLeNet5()
model.to(device)

checkpoint = torch.load('models/bayeslenet5.pt')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [3]:
n = 100
n_samples_list = [1, 5, 10, 50, 100, 500, 1000]

accuracies = []
pbar1 = tqdm(total=len(n_samples_list), leave=False)
for n_samples in n_samples_list:
    pbar1.set_description(f"n. samples: {n_samples}")
    
    acc = []
    pbar2 = tqdm(total=n, leave=False)
    for _ in range(n):    
        with torch.no_grad():
            pbar3 = tqdm(total=len(val_loader), leave=False)
            correct = 0
            for i, (input, target) in enumerate(val_loader):
                input = input.to(device)
                target = target.to(device)
                samples = torch.empty((n_samples, input.shape[0], 10)).to(device)
                for j in range(n_samples):
                    samples[j, ...] = F.softmax(model(input), dim=1)
                _, labels = samples.mean(dim=0).max(1)
                correct += labels.eq(target).sum()
                pbar3.update(1)
            pbar3.close()

        acc.append(correct.cpu().item() * 100. / len(val_dataset))
        pbar2.update(1)
    pbar2.close()
    
    accuracies.append(acc)
    pbar1.update(1)
pbar1.close()

pd.DataFrame(
    np.array(accuracies).T,
    columns=n_samples_list
).to_csv(
    'sample_accuracies.csv',
    index=False
)

In [10]:
df = pd.read_csv('sample_accuracies.csv')
df

,1,5,10,50,100,500,1000
0,98.02,98.77,98.89,98.98,99.00,99.00,98.97
1,97.85,98.77,98.96,98.97,99.03,99.03,99.00
2,98.05,98.96,98.92,99.05,98.99,99.01,99.02
3,97.81,98.75,99.00,98.97,98.98,99.00,99.03
4,97.79,98.67,98.87,99.09,98.99,99.01,99.03
...,...,...,...,...,...,...,...
95,97.92,98.90,98.92,99.04,98.97,99.06,99.00
96,97.85,98.86,98.99,99.01,99.05,98.99,98.99
97,97.79,98.91,98.88,99.03,98.98,99.00,98.98
98,97.81,98.89,98.96,99.01,98.99,98.99,98.99
